In [88]:
# clean and combine OCR text data in excel
import os
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

from dev import SUFFIXES
from data_cleaning_functions import remove_punc, clean_names


OCR_DIR = '/home/lraymond/Dropbox (MIT)/Christine/OCR'
FILE_FORMAT = '{year}_excel.xlsx'
YEAR_RANGE = (1966, 1977)
OCR_SUFFIXES = ['OR' ]

time: 3.57 ms


In [17]:
%load_ext autoreload
%autoreload 2

%load_ext autotime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.66 ms


In [24]:
year=1966

# create a file object from excel file
f1 = pd.ExcelFile(os.path.join(OCR_DIR, FILE_FORMAT.format(year=str(year))))
sheet_names = [f for f in f1.sheet_names if f.startswith('Sheet')]
print sheet_names

[u'Sheet1', u'Sheet2', u'Sheet3', u'Sheet4', u'Sheet5', u'Sheet6', u'Sheet7', u'Sheet8', u'Sheet9', u'Sheet10', u'Sheet11', u'Sheet12', u'Sheet13', u'Sheet14', u'Sheet15', u'Sheet16', u'Sheet17', u'Sheet18', u'Sheet19', u'Sheet20', u'Sheet21', u'Sheet22', u'Sheet23', u'Sheet24', u'Sheet25', u'Sheet26', u'Sheet27', u'Sheet28', u'Sheet29', u'Sheet30', u'Sheet31', u'Sheet32', u'Sheet33', u'Sheet34', u'Sheet35', u'Sheet36', u'Sheet37', u'Sheet38', u'Sheet39', u'Sheet40', u'Sheet41', u'Sheet42', u'Sheet43', u'Sheet44', u'Sheet45', u'Sheet46', u'Sheet47', u'Sheet48', u'Sheet49', u'Sheet50', u'Sheet51', u'Sheet52', u'Sheet53', u'Sheet54', u'Sheet55', u'Sheet56', u'Sheet57', u'Sheet58', u'Sheet59', u'Sheet60', u'Sheet61', u'Sheet62', u'Sheet63', u'Sheet64', u'Sheet65', u'Sheet66', u'Sheet67', u'Sheet68', u'Sheet69', u'Sheet70', u'Sheet71', u'Sheet72', u'Sheet73', u'Sheet74', u'Sheet75', u'Sheet76', u'Sheet77', u'Sheet78', u'Sheet79', u'Sheet80', u'Sheet81', u'Sheet82', u'Sheet83', u'Sheet84', 

In [34]:
s1 = f1.parse(sheet_names[0], header=None, names=['NAME'])



time: 3.74 ms


In [37]:
# first row has multiple names in the first row, need to parse

def has_multiple_names(row):
    # check if new lines present in row
    # this could indicate multiple names in the same row
    lst_names = row['NAME'].split('\n')
    return len(lst_names) > 1
    

time: 2.41 ms


In [ ]:
# pair word with previous word
# check check with first last name
# first, check breaks for 1 letter words or suffixes (first suffixes, then 1 letter words)
# if similarity is low, continue to next word

In [87]:
def is_last_name(last_str, first_last_name):
    print 'last name %s' % last_str
    print 'first last name %s' % first_last_name
    ratio = fuzz.ratio(first_last_name[:4], last_str[:4])
    print ratio
    if len(last_str) < 3:
        print last_str
    if ratio > 60 and first_last_name[0]==last_str[0]:
        return True
    return False

def is_one_name(name_str):
    wrds = mapl(lambda x: x.upper(), name_str.split(' '))
    if len(wrds) < 4 and len(wrds[-1]) == 1:
        return True
    last_wrd = remove_punc(wrds[-1].strip())
    if last_wrd in SUFFIXES+OCR_SUFFIXES:
        return True
    return False

def parse_multi_names(row):
    # parse multiple names from a row
    sep_names = []
    first_pass = row['NAME'].split('\n')
    return pd.DataFrame({'NAME': first_pass})
   
def correct_ocr_issues(raw_str):
    # strip spaces and punctuation
    COMMON_PROBLEMS = {'OR': 'DR', '8': 'B'}
    new_str = clean_names(raw_str.strip())
    for problem, correction in COMMON_PROBLEMS.iterkeys():
        if problem in new_str:
            new_str = new_str.replace(correction)
    return new_str 
    
    

time: 13.9 ms


In [69]:
multi_name_mask = s1.apply(has_multiple_names, axis=1)

new_names = s1[multi_name_mask].apply(parse_multi_names, axis=1)

time: 7.62 ms


In [76]:
test_names = 'AARONSON CHERYL AARONSON MAY R AARHUS ROBERT T ABBAMONTE PATRICIA A'

time: 1.23 ms


In [48]:
pd.DataFrame(new_names.values[0])

,0
0,AABEL JUDITH A
1,AANDAHL VIRGINIA
2,AARONSON CHERYL AARONSON MAY R AARHUS ROBERT T...
3,ABBATICCHIO TONI A


time: 9.86 ms
